In [ ]:
import copy
import random
from types import SimpleNamespace

import vrplib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd

from alns import ALNS
from alns.accept import RecordToRecordTravel
from alns.select import RouletteWheel
from alns.stop import MaxIterations

from myvrplib import cost_matrix_from_coords, read_cordeau_data, plot_data, plot_solution, calculate_depots, \
verify_time_windows, close_route

In [ ]:
%matplotlib inline
SEED = 1234

# Implementation of wang 2024
https://www.sciencedirect.com/science/article/pii/S0360835224002432?via%3Dihub

NOTE: data['dimension'] is the number of customers only, not including depots

In [ ]:
data = read_cordeau_data("./data/c-mdvrptw/pr02", print_data=True)

In [ ]:
print(data['depots'])

In [ ]:
print(data["demand"])
print(type(data["demand"]))

In [ ]:
data['node_coord'][0]

In [ ]:
print(len(data["service_time"]))
print(data['service_time'])

In [ ]:
plot_data(data)

## Solution state

In [ ]:
class CvrptwState:
    """
    Solution state for CVRPTW. It has two data members, routes and unassigned.
    Routes is a list of list of integers, where each inner list corresponds to
    a single route denoting the sequence of customers to be visited. A route
    does not contain the start and end depot. Times is a list of list, containing
    the planned arrival times for each customer in the routes. The outer list
    corresponds to the routes, and the inner list corresponds to the customers of
    the route. Unassigned is a list of integers, each integer representing an 
    unassigned customer.
    """

    def __init__(self, routes, times, unassigned=None):
        self.routes = routes
        self.times = times     # planned arrival times for each customer
        self.unassigned = unassigned if unassigned is not None else []

    def copy(self):
        return CvrptwState(
            copy.deepcopy(self.routes), self.times.copy(), self.unassigned.copy()
        )

    def objective(self):
        """
        Computes the total route costs.
        """
        return sum(route_cost(route) for route in self.routes)

    @property
    def cost(self):
        """
        Alias for objective method. Used for plotting.
        """
        return self.objective()

    def find_route(self, customer):
        """
        Return the route that contains the passed-in customer.
        """
        for route in self.routes:
            if customer in route:
                return route
        # print(f"Solution does not contain customer {customer}.")
    
    def find_index_in_route(self, customer, route):
        """
        Return the index of the customer in the route.
        """
        if customer in route:
            return route.index(customer)

        raise ValueError(f"Given route does not contain customer {customer}.")

    def update_times(self):
        """
        Update the arrival times of each customer in the routes.
        """
        for idx, route in enumerate(self.routes):
            self.times[idx] = self.evaluate_times_of_route(route)

    def evaluate_times_of_route(self, route):
        """
        Update the arrival times of each customer in a given route.
        """
        timings = [0]
        current_position = 0
        for customer in route:
            movement_time = data["edge_weight"][current_position][customer]
            # add the service time of the last customer
            timings.append(timings[-1] + data["service_time"][current_position])
            # add the movement time to reach next customer
            timings[-1] = float(timings[-1] + movement_time)

        return timings

    def print_state_dimensions(self):
        print(f"Number of routes: {len(self.routes)}")
        print(f"Length of routes: {[len(route) for route in self.routes]}")
        print(f"Dimensions of times: {[len(route) for route in self.times]}")
        print(f"Number of unassigned customers: {len(self.unassigned)}")
        print(f"Number of customers in routes: {sum(len(route) for route in self.routes)}")


# NOTE: maybe add time influence on cost of solution ?
def route_cost(route):
    distances = data["edge_weight"]
    tour = [0] + route + [0]

    return sum(
        distances[tour[idx]][tour[idx + 1]] for idx in range(len(tour) - 1)
    )

In [ ]:
def get_customer_info(data, state: CvrptwState, idx):
    """
    Get the customer information for the passed-in index.
    """
    route = state.find_route(idx)
    index_in_route = state.find_index_in_route(idx, route)
    route_index = state.routes.index(route)
    print(f"index_in_route: {index_in_route}")
    print(f"route: {route}")
    planned_service_time = state.times[route_index][index_in_route]

    dict = {
        "index": idx,
        "coords": data["node_coord"][idx],
        "demand": data["demand"][idx].item(),
        "ready time": data["time_window"][idx, 0].item(),
        "due time": data["time_window"][idx, 1].item(),
        "service_time": data["service_time"][idx].item(),
        "route": route,
        "planned service time": planned_service_time,
    }

    return dict

## Destroy operators

Destroy operators break parts of a solution down, leaving an incomplete state. This is the first part of each iteration of the ALNS meta-heuristic; the incomplete solution is subsequently repaired by any one repair operator. We will consider one destroy operator: **random removal**. We will also use a separate parameter, the degree of destruction, to control the extent of the damage done to a solution in each step.

In [ ]:
degree_of_destruction = 0.05
customers_to_remove = int((data["dimension"] - 1) * degree_of_destruction)
print(f"Removing {customers_to_remove} customers.")

In [ ]:
def random_removal(state, rng):
    """
    Removes a number of randomly selected customers from the passed-in solution.
    """
    destroyed = state.copy()

    for customer in rng.choice(
        range(data["dimension"]), customers_to_remove, replace=False
    ):
        destroyed.unassigned.append(customer)
        route = destroyed.find_route(customer)
        if route is not None:
            route.remove(customer)

    # NOTE: now evaluate the time of the modified routes and return them
    destroyed.update_times()

    return remove_empty_routes(destroyed)


def remove_empty_routes(state):
    """
    Remove empty routes and timings after applying the destroy operator.
    """
    routes_idx_to_remove = [idx for idx, route in enumerate(state.routes) if len(route) == 0]
    state.routes = [route for idx, route in enumerate(state.routes) if idx not in routes_idx_to_remove] 
    state.times = [timing for idx, timing in enumerate(state.times) if idx not in routes_idx_to_remove]
    state.times = [timing for idx, timing in enumerate(state.times) if len(state.routes[idx]) != 0]
    return state

In [ ]:
def time_window_compatibility(tij, twi: tuple, twj: tuple):
    """
    Time Window Compatibility (TWC) between a pair of vertices i and j. Base on
    'An adaptive large neighborhood search for the multi-depot dynamic vehicle
    routing problem with time windows' by Wang et al. (2024)
    """
    (ai, bi) = twi
    (aj, bj) = twj
    if bj > ai +tij:
        return -np.inf  #Incompatible time windows
    else:
        min([bi + tij, bj]) - max([ai + tij, aj])
        

In [ ]:
def cost_reducing_removal(state, rng):
    """
    Cost reducing removal operator based on 'An adaptive large neighborhood search for the multi-depot dynamic vehicle routing problem with time windows'.
    """
    destroyed = state.copy()

    for first_route in rng.choice(range(state.routes), 1):
        for v in first_route:
            i1 = first_route[first_route.index(v) + 1]   #previous customer
            j1 = first_route[first_route.index(v) - 1]   #next customer
            di1v = data["edge_weight"][i1][v]
            dvj1 = data["edge_weight"][v][j1]
            di1j1 = data["edge_weight"][i1][j1]
            
            for second_route in state.routes:
                for i2 in second_route:     #first customer
                    time_window_compatibility()
                    j2 = second_route[second_route.index(i2) + 1]   #second customer
                    di2j2 = data["edge_weight"][i2][j2]
                    di2v = data["edge_weight"][i2][v]
                    dvj2 = data["edge_weight"][v][j2]

                    if di1v + dvj1 + di2j2 > di1j1 + di2v + dvj2:
                        first_route.remove(v)
                        second_route.insert(second_route.index(i2), v)
                        destroyed.update_times()

    return remove_empty_routes(destroyed)

## Repair operators
We implement a simple, **greedy repair** strategy. It iterates over the set of unassigned customers and finds the best route and index to insert to, i.e., with the least increase in cost. Time window constraints are implemented as follows in the **greedy repair**:

1)  Within the set of unvisited nodes, first find those nodes that can be visited within the node's time window from the current end of each existing route
2)  Add the node that is nearest to the end of some route to that route
3)  If no unvisited nodes can be visited on time, make a new route just for that node

In [ ]:
def greedy_repair(state, rng):
    """
    Inserts the unassigned customers in the best route. If there are no
    feasible insertions, then a new route is created. Only checks capacity constraints.
    """
    rng.shuffle(state.unassigned)

    while len(state.unassigned) != 0:
        customer = state.unassigned.pop()
        route, idx = best_insert(customer, state)

        if route is not None:
            route.insert(idx, customer.item())
            state.update_times()
            # DEBUG
            # print(f"\nCustomer {customer} inserted in route {route} at position {idx}.")
            # print(f"Planned service time for customer {customer}: {state.times[state.routes.index(route)][idx]}")
            # print(f"Requested time window: {data['time_window'][customer]}\n")
        else:
            # Initialize a new route and corresponding timings
            # Check if the number of routes is less than the number of vehicles
            if len(state.routes) < data["vehicles"]:
                state.routes.append([customer.item()])
                state.times.append([0])
                state.routes[-1] = close_route(state.routes[-1])
                state.update_times()    # NOTE: maybe not needed
            # else:
            #     print(f"Customer {customer} could not be inserted in any route. Maximum number of routes/vehicles reached.")

    return state

def greedy_repair_tw(state, rng):
    """
    Inserts the unassigned customers in the best route. If there are no
    feasible insertions, then a new route is created. Check capacity and time window constraints.
    """
    rng.shuffle(state.unassigned)

    while len(state.unassigned) != 0:
        customer = state.unassigned.pop()
        route, idx = best_insert_tw(customer, state)

        if route is not None:
            route.insert(idx, customer.item())
            state.update_times()
        else:
            # Initialize a new route and corresponding timings
            # Check if the number of routes is less than the number of vehicles
            if len(state.routes) < data["vehicles"]:
                state.routes.append([customer.item()])
                state.times.append([0])
                state.update_times()    # NOTE: maybe not needed
            # else:
                # print(f"Customer {customer} could not be inserted in any route. Maximum number of routes/vehicles reached.")

    return state


def best_insert(customer, state):
    """
    Finds the best feasible route and insertion idx for the customer.
    Return (None, None) if no feasible route insertions are found.
    Only checks capacity constraints.
    """
    best_cost, best_route, best_idx = None, None, None

    for route_number, route in enumerate(state.routes):
        for idx in range(1, len(route)):
        # for idx in range(1, len(route)+1):

            # DEBUG
            # print(f"In best_insert: route_number = {route_number}, idx = {idx}")
            if can_insert(customer, route_number, idx, state):
                cost = insert_cost(customer, route, idx)             

                if best_cost is None or cost < best_cost:
                    best_cost, best_route, best_idx = cost, route, idx

    return best_route, best_idx


def best_insert_tw(customer, state):
    """
    Finds the best feasible route and insertion idx for the customer.
    Return (None, None) if no feasible route insertions are found.
    Checks both capacity and time window constraints.
    """
    best_cost, best_route, best_idx = None, None, None

    for route_number, route in enumerate(state.routes):
        # for idx in range(1, len(route) + 1):
        for idx in range(1, len(route)):

            # DEBUG
            # print(f"In best_insert: route_number = {route_number}, idx = {idx}")
            if can_insert_tw(customer, route_number, idx, state):
                cost = insert_cost(customer, route, idx)

                if best_cost is None or cost < best_cost:
                    best_cost, best_route, best_idx = cost, route, idx

    return best_route, best_idx


# NOTE: I think performance can be improved by changing this function
def can_insert(customer, route_number, idx, state):
    """
    Checks if inserting customer in route 'route_number' at position 'idx' does not exceed vehicle capacity.
    """
    route = state.routes[route_number]

    # Capacity check
    total = data["demand"][route].sum() + data["demand"][customer]
    if total > data["capacity"]:
        return False
    return True


# NOTE: I think performance can be improved by changing this function
def can_insert_tw(customer, route_number, idx, state):
    """
    Checks if inserting customer in route 'route_number' at position 'idx' does not exceed vehicle capacity and time window constraints.
    """

    route = state.routes[route_number]
    # Capacity check
    total = data["demand"][route].sum() + data["demand"][customer]
    if total > data["capacity"]:
        return False
    # Time window check
    if time_window_check(state.times[route_number][idx - 1], route[idx - 1], customer):
        return route_time_window_check(route, state.times[route_number])


def route_time_window_check(route, times):
    """
    Check if the route satisfies time-window constraints. Ignores the depots as
    they are considered available 24h.
    """
    route = route[1:] # Ignore the depot
    for idx, customer in enumerate(route):
        if times[idx] > data["time_window"][customer][1]:
            return False

    return True

# NOTE: this is a terrible check.
# It will accept any customer whose time window is after the calculated arrival time,
# even if the vehicle is early.
# Is the vehicle allowd to be early?
def time_window_check(prev_customer_time, prev_customer, candidate_customer):
    """
    Check if the candidate customer satisfies time-window constraints. Ignores the depots as
    they are considered available 24h.
    """
    return prev_customer_time + data["service_time"][prev_customer] + data[
        "edge_weight"][prev_customer][candidate_customer] <= data["time_window"][candidate_customer][1]

def insert_cost(customer, route, idx):
    """
    Computes the insertion cost for inserting customer in route at idx.
    """
    dist = data["edge_weight"]
    pred = 0 if idx == 0 else route[idx - 1]
    succ = 0 if idx == len(route) else route[idx]

    # Increase in cost of adding customer, minus cost of removing old edge
    return dist[pred][customer] + dist[customer][succ] - dist[pred][succ]

## Initial solution
We need an initial solution that is going to be destroyed and repaired by the ALNS heuristic. To this end, we use a simple *nearest neighbor (NN)* heuristic. NN starts with an empty solution and iteratively adds the nearest customer to the routes. If there are no routes available, then a new route is created.

### Choosing starting depot
If the number of vehicles if larger than number of depots we split the number of vehicles between the depots.

Otherwise, we choose randomly a depot and generate a route from it.
NOTE: maybe performance of the model can be improved by changing the above policy

In [ ]:
calculate_depots(data)
print(data['depot_to_vehicles'])
print(data['vehicle_to_depot'])
print(data['dimension'])

In [ ]:
def neighbors(customer, depots: list = []):
    """
    Return the nearest neighbors of the customer, excluding the depot.
    """
    locations = np.argsort(data["edge_weight"][customer])
    return [loc for loc in locations if loc not in depots]

def nearest_neighbor_tw():
    """
    Build a solution by iteratively constructing routes, where the nearest
    time-window compatible customer is added until the route has met the 
    vehicle capacity limit.
    """
    routes = []
    full_schedule = []
    unvisited = set(range(data["dimension"]))
    vehicle = 0

    while unvisited and vehicle < data["vehicles"]:
        # Mapping vehicle i to depot i
        calculate_depots(data)

        initial_depot = data["vehicle_to_depot"][vehicle]
        route = [initial_depot]  # Start at the depot
        route_schedule = [0]
        route_demands = 0
        print(f"Route of vehicle {vehicle} starts at depot {initial_depot}")
        while unvisited:
            # Add the nearest unvisited customer to the route till max capacity
            current = route[-1]
            # print(f"current = {current}")
            # print(
            #     f"len(neighbors({current})), len(unvisited) = {len(neighbors(current, depots=data['depots']))}, {len(unvisited)}"
            # )
            # print(f"unvisited = {unvisited}")
            nearest = [nb for nb in neighbors(current, depots=data['depots']) if nb in unvisited][0]
            nearest = int(nearest)
            # print(f"nearest = {nearest}")

            if route_demands + data["demand"][nearest] > data["capacity"]:
                break

            if not time_window_check(route_schedule[-1], current, nearest):
                break
            if not route_time_window_check(route, route_schedule):
                break

            route.append(nearest)
            route_schedule.append(
                data["edge_weight"][current][nearest].item()
                + data["service_time"][current].item()
            )

            unvisited.remove(nearest)
            route_demands += data["demand"][nearest]

        # customers = route[1:]  # Remove the depot
        # routes.append(customers)
        routes.append(route)
        full_schedule.append(route_schedule)
        # Consider new vehicle
        vehicle += 1

    if unvisited:
        print(f"Unvisited customers: {unvisited}")
    if vehicle < data["vehicles"]:
        print(f"Vehicles left: {data['vehicles'] - vehicle}")

    return CvrptwState(routes, full_schedule)

In [ ]:
initial_solution = nearest_neighbor_tw()

plot_solution(data, initial_solution, "Nearest neighbor solution")

In [ ]:
for route in initial_solution.routes:
    print(route)

In [ ]:
late, early, ontime, left_out = verify_time_windows(data, initial_solution)
print(f"Late: {late}, Early: {early}, Ontime: {ontime}")
print(f"Left out customers because of not enough routes/vehicles: {left_out}")

## Heuristic solution

Let's now construct our ALNS heuristic. Since we only have one destroy and repair operator, we do not actually use any adaptive operator selection -- but you can easily add more destroy and repair operators. 

In [ ]:
alns = ALNS(rnd.default_rng(SEED))

alns.add_destroy_operator(random_removal)
alns.add_destroy_operator(cost_reducing_removal)

alns.add_repair_operator(greedy_repair)

In [ ]:
num_iterations = 5000
init = nearest_neighbor_tw()
select = RouletteWheel([25, 5, 1, 0], 0.8, 1, 1)
accept = RecordToRecordTravel.autofit(
    init.objective(), 0.02, 0, num_iterations
)
stop = MaxIterations(num_iterations)
result = alns.iterate(init, select, accept, stop)

In [ ]:
solution = result.best_state
objective = solution.objective()
print(f"Best heuristic objective is {objective}.")

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))
result.plot_objectives(ax=ax)

In [ ]:
plot_solution(data, initial_solution, "Nearest-neighbor-solution", save=True, figsize=(8, 8))
plot_solution(data, solution, "Heuristic-solution", idx_annotations=False, save=True, figsize=(8, 8))

**Nearest neighbours**: ![](./plots/Nearest-neighbor-solution.png)  **Heuristic**: ![](./plots/Heuristic-solution.png)

In [ ]:
for route in solution.routes:
    print(route)